In [1]:
#Import Packages
import pandas as pd
import os
import re
import datetime
import random
import re

In [2]:
from nltk.corpus import stopwords
from nltk import word_tokenize

Stop_Words = set(stopwords.words('english')+['"'])

In [3]:
#!pip install pdftotree
#!pip install pypdf2
#import PyPDF2 as pdf2
import pdftotree as pdf3
from bs4 import BeautifulSoup

In [4]:
File_names=os.listdir('../world_bank_loans_full_upload-20210511T002631Z-001/world_bank_loans_full_upload/')

In [5]:
#Create a data frame with information from the files names
def get_info_file_name(file_name):
    file_split=file_name.split('--')
    date=datetime.datetime.strptime(' '.join(file_split[0].split('_')[0:3]),'%Y %B %d')
    try:
        title= file_split[2]
        type_contract=f.split('--')[3].replace('.pdf','')
    except:
        title=type_contract=''
    return [file_name,date,title,type_contract]


In [6]:
df = pd.DataFrame(columns=('File_Name', 'Date','Project Title','Type','Borrower','Bank','Currency','Amount'))
for i in range(len(File_names)):
    f=File_names[i]
    df.loc[i] = get_info_file_name(f)+['','','',0]

In [7]:
df

,File_Name,Date,Project Title,Type,Borrower,Bank,Currency,Amount
0,1990_april_24_668811468165272290_conformed-cop...,1990-04-24,water-supply-project,loan-agreement,,,,0
1,1990_april_25_904191468298750561_conformed-cop...,1990-04-25,environment-management-project,loan-agreement,,,,0
2,1990_april_30_725911468042268845_conformed-cop...,1990-04-30,third-telecommunications-project,loan-agreement,,,,0
3,1990_august_10_460651468271855106_conformed-co...,1990-08-10,universities-science-and-technology-research-p...,loan-agreement,,,,0
4,1990_august_10_939121468273631507_conformed-co...,1990-08-10,juam-regional-water-supply-project,loan-agreement,,,,0
...,...,...,...,...,...,...,...,...
2224,2019_september_13_300871570120923592_official-...,2019-09-13,,,,,,0
2225,2019_september_13_710891569417913880_official-...,2019-09-13,,,,,,0
2226,2019_september_23_867961572361092133_official-...,2019-09-23,,,,,,0
2227,2019_september_25_810411569965213165_official-...,2019-09-25,,,,,,0


In [8]:
#Code to work with long paths without problem 
#Taken from: https://stackoverflow.com/a/63198228/3254178
def winapi_path(dos_path, encoding=None):
    if (not isinstance(dos_path, str) and encoding is not None): 
        dos_path = dos_path.decode(encoding)
    path = os.path.abspath(dos_path)
    if path.startswith(u"\\\\"):
        return u"\\\\?\\UNC\\" + path[2:]
    return u"\\\\?\\" + path

In [9]:
#Get Text from header
def get_element_text(element):
    words=[w.text.lower() for w in element.find_all('span',{'class':'ocrx_word'})]
    text=' '.join(words)
    numbers_in_text = re.findall("^(?:[+-]|\()?\$?\d+(?:,\d+)*(?:\.\d+)?\)?$", text)
    for num in numbers_in_text:
        text = text.replace(num, ' '+num+' ')
    words=word_tokenize(text)
    words=[w for w in words if not w in Stop_Words]
    return words

def get_element_class(element):
    return element['pdftotree']  

In [10]:
#Convert each pdf document into html and use Beautifulsoup and Regex to extract the desired data. 

def get_borrower_bank_amount_file(path):
    #Read PDF using pdftotree
    try:
        html_doc=pdf3.parse(path, html_path=None, model_type=None, model_path=None, visualize=False)
    except:
        return '','','',0

    #Convert into a BeautifulSoup Object
    soup = BeautifulSoup(html_doc,'html.parser')

    #Most important info is in ocrx_blocks that are section_headers or paragraphs, so we extract them.
    headers_paragraphs=soup.find_all('div',{'class':"ocrx_block", 'pdftotree':["section_header","paragraph"]})

    #Organize this information in a pandas DataFrame
    Document_df = pd.DataFrame(columns=('Element','Element_Class', 'Text'))
    for i in range(len(headers_paragraphs)):
        element=headers_paragraphs[i]
        Document_df.loc[i] = [element,get_element_class(element),get_element_text(element)]
        
    #Find LOAN AGREEMENT
    for index_loan in Document_df.index:
        if ('loan' in Document_df['Text'][index_loan]) and ('agreement' in Document_df['Text'][index_loan]) and \
        (Document_df['Element_Class'][index_loan]=='section_header'):
            break
    
    try:
        Borrower_Loaner_Text=' '.join(Document_df['Text'][index_loan+1])
        Borrower=re.findall('([\w|\s|.]+)\W+ borrower',Borrower_Loaner_Text)[0]
        Bank=re.findall('([\w|\s|.]+)\W+bank',Borrower_Loaner_Text)[0]
    except:
        Borrower=Bank=''
    
    #Find Amount Borrowed
    for index_article in Document_df.index:
        if ('bank' in Document_df['Text'][index_article]) and ('agrees' in Document_df['Text'][index_article]) and \
        ('amount' in Document_df['Text'][index_article]) and(Document_df['Element_Class'][index_article]=='paragraph'):
            break
    try:
        Value_Borrowed_Text=' '.join(Document_df['Text'][index_article])
        
        #Extract the values using Regular Expressions
        currency_characters=u"[$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6]|"
        currency_abbs=u'aed|afn|all|amd|ang|aoa|ars|aud|awg|azn|bam|bbd|bdt|bgn|bhd|bif|bmd|bnd|bob|bov|brl|bsd|btn|bwp|byn|bzd|cad|cdf|che|chf|chw|clf|clp|cny|cop|cou|crc|cuc|cup|cve|czk|djf|dkk|dop|dzd|egp|ern|etb|eur|fjd|fkp|gbp|gel|ghs|gip|gmd|gnf|gtq|gyd|hkd|hnl|hrk|htg|huf|idr|ils|inr|iqd|irr|isk|jmd|jod|jpy|kes|kgs|khr|kmf|kpw|krw|kwd|kyd|kzt|lak|lbp|lkr|lrd|lsl|lyd|mad|mdl|mga|mkd|mmk|mnt|mop|mru|mur|mvr|mwk|mxn|mxv|myr|mzn|nad|ngn|nio|nok|npr|nzd|omr|pab|pen|pgk|php|pkr|pln|pyg|qar|ron|rsd|rub|rwf|sar|sbd|scr|sdg|sek|sgd|shp|sll|sos|srd|ssp|stn|svc|syp|szl|thb|tjs|tmt|tnd|top|try|ttd|twd|tzs|uah|ugx|usd|usn|uyi|uyu|uyw|uzs|ves|vnd|vuv|wst|xaf|xag|xau|xba|xbb|xbc|xbd|xcd|xdr|xof|xpd|xpf|xpt|xsu|xts|xua|xxx|yer|zar|zmw|zwl'
        value=re.findall('('+currency_characters+currency_abbs+') (\d{1,3}[,\d{3}]*[\.\d]*)',Value_Borrowed_Text)

        Currency=value[0][0]
        Amount=float(value[0][1].replace(',',''))
    except:
        Currency=''
        Amount=0         
    return Borrower, Bank, Currency, Amount

In [11]:
#Take just a sample of the Files
df=df.sample(10)

In [12]:
ROOT_DIR = '../world_bank_loans_full_upload-20210511T002631Z-001/world_bank_loans_full_upload/'

for i in df.index:
    file=File_names[i]
    path=os.path.abspath(ROOT_DIR+file)
    path=winapi_path(path)
    try:
        Borrower, Bank, Currency, Amount=get_borrower_bank_amount_file(path)
        df.loc[i] = get_info_file_name(file)+[Borrower,Bank,Currency,Amount]
    except:
        df.loc[i] = get_info_file_name(file)+['','','',0]

Got stderr: May 12, 2021 9:52:08 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Got stderr: May 12, 2021 9:52:09 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Got stderr: May 12, 2021 9:54:10 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Got stderr: May 12, 2021 9:54:11 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [13]:
pd.set_option('display.max_rows', None)

In [14]:
df

,File_Name,Date,Project Title,Type,Borrower,Bank,Currency,Amount
1632,2014_august_28_230011468068332650_official-doc...,2014-08-28,,,quot,international,,0
43,1990_november_7_123971468052804464_conformed-c...,1990-11-07,,,,,$,1.2e+07
1290,2010_july_6_255061468109479670_loan-agreement-...,2010-07-06,,,0,international bank reconstruction development,$,2e+08
1276,2010_july_19_201181468024249523_loan-agreement...,2010-07-19,,,republic el salvador,international bank reconstruction development,,0
2031,2018_april_20_326651524256707376_official-docu...,2018-04-20,,,,,,0
1598,2013_november_5_200971468248974302_official-do...,2013-11-05,,,,,,0
1295,2010_june_10_288401468014434513_loan-agreement...,2010-06-10,,,entered federative republic brazil,international bank reconstruction development,$,1.3e+09
444,1995_january_27_130911468054232681_conformed-c...,1995-01-27,,,,,$,2.64e+07
837,2000_january_20_582701468333300909_conformed-c...,2000-01-20,,,,,,0
430,1995_february_6_111741468093272511_conformed-c...,1995-02-06,,,,,$,1.1e+08


In [15]:
file='1994_december_22_168961468248656295_egypt--irrigation-improvement-project-_-loan-3832--loan-agreement--conformed.pdf'
path=os.path.abspath(ROOT_DIR+file)
path=winapi_path(path)

In [19]:
html_doc=pdf3.parse(path, html_path=None, model_type=None, model_path=None, visualize=False)

FileNotFoundError: [Errno 2] No such file or directory: '\\\\?\\C:\\Users\\Francisco\\OneDrive - The Ohio State University\\Documents\\Summer 2021\\Amethyst Project\\world_bank_loans_full_upload-20210511T002631Z-001\\world_bank_loans_full_upload\\1994_december_22_168961468248656295_egypt--irrigation-improvement-project-_-loan-3832--loan-agreement--conformed.pdf'

In [1]:
import cv2

In [2]:
pil_image = PIL.Image.open('Image.jpg').convert('RGB') 
open_cv_image = numpy.array(pil_image)

NameError: name 'd' is not defined